**ANTES DE PROSSEGUIR FAÇA UMA CÓPIA DESTE NOTEBOOK**

# SI10 | 2025 | T10 | Prova Prática



```
# Isto está formatado como código
```

**Nome**: *Nicollas Isaac Queiroz Batista*

----

## Contexto

Você foi contratado como analista de dados por uma empresa de e-commerce que está enfrentando desafios na conversão de visitantes em clientes.

> Adicionar aspas



Ao investigar os registros de navegação, a equipe de produto identificou gargalos no fluxo de páginas e propôs uma alteração estratégica no site no dia `2023-01-30`, com o objetivo de melhorar a taxa de conversão.

Seu trabalho consiste em analisar os dados de navegação antes e depois da intervenção, identificar padrões de comportamento dos usuários e responder se a mudança surtiu efeito.

## Importação do dataset e das bibliotecas

In [2]:
# Execute essas linhas de código

# Importação do dataset

!gdown 1gAe0H_gv_rk_9wPnBgB1rzTHi_nnkpDc

Downloading...
From: https://drive.google.com/uc?id=1gAe0H_gv_rk_9wPnBgB1rzTHi_nnkpDc
To: /content/ITL-SI10-2025-Prova_Dataset.csv
100% 178k/178k [00:00<00:00, 84.7MB/s]


In [3]:
# Execute essas linhas de código

# Importação das bibliotecas
import pandas as pd

# Transforma o dataset em DataFrame Pandas
df = pd.read_csv('ITL-SI10-2025-Prova_Dataset.csv')
print(df.head())

   user_id session_id        data  passo   pagina
0      956     00_000  2023-01-01      1     Home
1      956     00_000  2023-01-01      2  Produto
2      956     00_000  2023-01-01      3    Saída
3      418     00_001  2023-01-01      1     Home
4      418     00_001  2023-01-01      2    Saída


## Questões

In [13]:
# Execute essas linhas de código

dia_mudanca = 30
df["data"] = pd.to_datetime(df["data"])
data_mudanca = pd.Timestamp("2023-01-01") + pd.Timedelta(days=dia_mudanca)

### Questão 1

**Matriz de Transição de Markov**. A empresa deseja entender como os usuários se comportavam antes da mudança no site. Para isso, utilize o conjunto de dados `df_antes`, que contém os registros de navegação anteriores ao dia `2023-01-30`.

Construa uma matriz de transição de Markov onde cada célula representa a probabilidade de um usuário transitar de uma página para outra dentro da mesma sessão.

In [ ]:
from abc import abstractstaticmethod
# Execute essas linhas de código
import numpy as np


df_antes = df[df["data"] < data_mudanca].copy()
df_antes.head()

estados_site = ["Home", "Produto", "Saída", "Obrigado"]

# Contar transições
contagem_transicoes = pd.DataFrame(0, index=df_antes, columns=df_antes)

for jornada in df_antes['pagina']:
    for i in range(len(jornada) - 1):
        estado_atual = jornada[i]
        proximo_estado = jornada[i+1]
        contagem_transicoes.loc[estado_atual, proximo_estado] += 1

# Normalizar para obter probabilidades
matriz_transicao_calculada = contagem_transicoes.div(contagem_transicoes.sum(axis=1), axis=0).fillna(0)

print("\nMatriz de Transição calculada a partir de dados hipotéticos:")
print(matriz_transicao_calculada)

df_antes.head()

### Questão 2

**Simulação de Monte Carlo**: Simule a navegação de 1.000 usuários iniciando pela página Home, utilizando a matriz de transição de Markov para determinar os próximos passos..

📌 Regras da simulação:

- Cada simulação começa no estado `Home`.

- A navegação termina quando o usuário chega a um estado final: `Obrigado` (conversão) ou `Saída` (abandono).

- Use sorteios aleatórios baseados nas probabilidades da matriz para definir as transições.


Entregável:

- Número de usuários que chegaram até Obrigado.

- Número de usuários que chegaram até Saída.

- Taxa de conversão e taxa de abandono estimadas com base na simulação.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parâmetros do investimento
valor_inicial = 1000
absorve_medio_diario = ["Obrigado", "Saída"]
desvio_padrao_diario = 0.01    # 1% de volatilidade diária
num_dias = 10          # 10 dias de simulação
num_simulacoes = 1000   # Número

# Array para armazenar os resultados finais de cada simulação
resultados_finais = []

plt.figure(figsize=(10, 6))

for _ in range(num_simulacoes):
    # Começar cada simulação com o valor inicial
    valores_diarios = [valor_inicial]
    valor_atual = valor_inicial

    for dia in range(num_dias):
        # Gerar um retorno diário aleatório a partir de uma distribuição normal
        # O retorno é modelado como (1 + absorve_medio_diario + erro_aleatorio)
        retorno_aleatorio = np.random.normal(absorve_medio_diario, desvio_padrao_diario)
        valor_atual *= (1 + retorno_aleatorio)
        valores_diarios.append(valor_atual)

    resultados_finais.append(valor_atual)
    plt.plot(valores_diarios, alpha=0.1, color=\'blue\
') # Plotar cada trajetória com baixa opacidade

# Converter resultados para um array numpy para facilitar cálculos
resultados_finais = np.array(resultados_finais)

# Calcular estatísticas dos resultados finais
valor_medio_final = np.mean(resultados_finais)
mediana_final = np.median(resultados_finais)
desvio_padrao_final = np.std(resultados_finais)
percentil_5 = np.percentile(resultados_finais, 5)
percentil_95 = np.percentile(resultados_finais, 95)

plt.title(\'Simulação de Monte Carlo de Valor de estado final (Obrigado e Saída)\
')
plt.xlabel(\'Dias\
')
plt.ylabel(\'Valor do Investimento (R$)\
')
plt.grid(True, linestyle=\'--\
', alpha=0.7)
plt.show()

# Histograma dos resultados finais
plt.figure(figsize=(8, 5))
plt.hist(resultados_finais, bins=50, edgecolor=\'black\
', alpha=0.7)
plt.axvline(valor_medio_final, color=\'red\
', linestyle=\'dashed\
', linewidth=1, label=f\'Média: R$ {valor_medio_final:,.2f}\
')
plt.axvline(percentil_5, color=\'green\
', linestyle=\'dashed\
', linewidth=1, label=f\'5º Percentil: R$ {percentil_5:,.2f}\
')
plt.axvline(percentil_95, color=\'purple\
', linestyle=\'dashed\
', linewidth=1, label=f\'95º Percentil: R$ {percentil_95:,.2f}\
')
plt.title(\'Distribuição dos Valores Finais do Investimento\
')
plt.xlabel(\'Valor Final do Investimento (R$)\
')
plt.ylabel(\'Frequência\
')
plt.legend()
plt.grid(True, linestyle=\'--\
', alpha=0.7)
plt.show()

In [21]:
# Data de início das mudanças
data_inicio_mudancas = pd.Timestamp('2023-01-30')

# Separar dados antes e depois das mudanças
df_depois = df[df['data'] >= data_inicio_mudancas]

# Calcular estatísticas para a taxa de rejeição e o volume de tráfego
media_antes_rejeicao = df_antes['passo'].mean()
media_depois_rejeicao = df_depois['passo'].mean()
std_antes_rejeicao = df_antes['passo'].std()
std_depois_rejeicao = df_depois['passo'].std()

media_antes_trafego =df_antes['passo'].mean()
media_depois_trafego = df_depois['passo'].mean()
std_antes_trafego = df_antes['passo'].std()
std_depois_trafego = df_depois['passo'].std()

# Função de simulação que respeita as variações históricas máximas e divide a contribuição das mudanças
def simular_impacto(n_simulacoes):
    resultados_rejeicao = []
    resultados_trafego = []

    for _ in range(n_simulacoes):
        contribuicao_ui = np.random.uniform(0, 1)
        contribuicao_velocidade = 1 - contribuicao_ui

        ajuste_rejeicao = (media_depois_rejeicao - media_antes_rejeicao) * contribuicao_ui
        ajuste_trafego = (media_depois_trafego - media_antes_trafego) * contribuicao_velocidade

        nova_taxa_rejeicao = np.random.normal(media_antes_rejeicao + ajuste_rejeicao, std_depois_rejeicao)
        novo_volume_trafego = np.random.normal(media_antes_trafego + ajuste_trafego, std_depois_trafego)

        resultados_rejeicao.append(nova_taxa_rejeicao)
        resultados_trafego.append(novo_volume_trafego)

    return resultados_rejeicao, resultados_trafego

# Executar simulação
n_simulacoes = 1000
resultados_rejeicao_simulada, resultados_trafego_simulado = simular_impacto(n_simulacoes)

print(simular_impacto(n_simulacoes))

([2.6484841983493417, 2.6401374843316416, 3.7488723126316867, 1.2864345206304804, 1.4440180266607494, 1.1932554553210877, 4.24634846801873, 1.815390773426025, 0.9858066833484058, 2.5815928050283636, 1.9168000397484286, 1.1394084061995968, 0.8461859716477542, 1.8809888805769466, 2.760471034064802, 2.062538843556315, 1.4391757389637427, 3.33127089132465, 1.7724479668012778, 3.1473781998325023, 1.5417861238327029, -0.007540561967028125, 4.855528202392544, 1.7317854758140667, 2.9101281199974682, 2.967084763429793, 4.09857662463469, 3.228423695784075, 2.7651787908667256, 1.362796134642396, 4.357232367473499, 2.456669319124497, 2.683131475468918, 1.2894455696928324, 1.430260078017537, 3.552532780067075, 1.837034293128463, 0.9447419643491834, 1.744967487977129, 2.785312755061314, 1.982592320525037, 2.8463473037875535, 0.496380128847979, 1.4563921547279808, 1.787117733576915, 2.74432074935645, 1.0589206657820303, 2.475017459628536, 1.539402032211339, 2.738774961883382, 1.960335858566269, 2.086

### Questão 3

**Teste A/B**: A mudança proposta no site foi implantada no dia `2023-01-30`. Para avaliar seu impacto, você recebeu dois subconjuntos de dados:

Você recebeu dois conjuntos de dados:

- `df_antes`: registros de navegação dos usuários antes da mudança

- `df_depois`: registros após a mudança

Verifique se a mudança foi eficaz em melhorar o comportamento dos usuários.

In [ ]:
df_depois = df[df["data"] >= data_mudanca].copy()
df_depois.head()

,user_id,session_id,data,passo,pagina
2874,551,30_000,2023-01-31,1,Home
2875,551,30_000,2023-01-31,2,Produto
2876,551,30_000,2023-01-31,3,Carrinho
2877,551,30_000,2023-01-31,4,Saída
2878,355,30_001,2023-01-31,1,Home


In [ ]:
# Função para simular a jornada de um único cliente
def simular_jornada_cliente(matriz_transicao, estados, estado_inicial=\"Home\", max_passos=20):
    jornada = [estado_inicial]
    estado_atual = estado_inicial

    # Mapear estados para índices numéricos para uso com numpy.random.choice
    estado_para_idx = {estado: i for i, estado in enumerate(estados)}
    idx_para_estado = {i: estado for i, estado in enumerate(estados)}

    estado_atual_idx = estado_para_idx[estado_inicial]

    for _ in range(max_passos):
        # Se o cliente chegou a um estado absorvente, a jornada termina
        if estado_atual in [\"Obrigado\", \"Saída\"]:
            break

        # Obter as probabilidades de transição para o estado atual
        probabilidades = matriz_transicao.loc[estado_atual].values

        # Escolher o próximo estado com base nas probabilidades
        proximo_estado_idx = np.random.choice(len(estados), p=probabilidades)
        proximo_estado = idx_para_estado[proximo_estado_idx]

        jornada.append(proximo_estado)
        estado_atual = proximo_estado
        estado_atual_idx = proximo_estado_idx

    return jornada

# Número de simulações de Monte Carlo
num_simulacoes_mc = 10000

resultados_jornadas = []
for _ in range(num_simulacoes_mc):
    jornada = simular_jornada_cliente(matriz_transicao_funil, estados_funil)
    resultados_jornadas.append(jornada[-1]) # Pegar o estado final da jornada

# Contar a frequência dos estados finais
contagem_estados_finais = pd.Series(resultados_jornadas).value_counts(normalize=True)

print("\nDistribuição dos Estados Finais dos Clientes (Simulação de Monte Carlo):")
print(contagem_estados_finais)

# Visualização da distribuição dos estados finais
plt.figure(figsize=(8, 5))
sns.barplot(x=contagem_estados_finais.index, y=contagem_estados_finais.values, palette=\"viridis\")
plt.title(\"Distribuição de Estados Finais de Clientes (Monte Carlo)\")
plt.xlabel(\"Estado Final\")
plt.ylabel(\"Proporção\")
plt.grid(axis=\"y\", linestyle=\"--\", alpha=0.7)
plt.show()

# Simular dados para o Teste A/B na transição Lead -> Qualificado
np.random.seed(123) # Para reprodutibilidade

# Grupo A (Controle)
leads_A = 1000
qualificados_A = np.random.binomial(n=1, p=0.60, size=leads_A).sum() # Taxa de qualificação de 60%

# Grupo B (Variante - Nova Estratégia)
leads_B = 1000
qualificados_B = np.random.binomial(n=1, p=0.65, size=leads_B).sum() # Taxa de qualificação de 65%

print(f"\n--- Resultados do Teste A/B (Lead -> Qualificado) ---")
print(f"Grupo A (Controle): {leads_A} Leads, {qualificados_A} Qualificados")
print(f"Grupo B (Variante): {leads_B} Leads, {qualificados_B} Qualificados")

taxa_qualificacao_A = qualificados_A / leads_A
taxa_qualificacao_B = qualificados_B / leads_B

print(f"Taxa de Qualificação Grupo A: {taxa_qualificacao_A:.4f}")
print(f"Taxa de Qualificação Grupo B: {taxa_qualificacao_B:.4f}")

# Realizar o teste Z para proporções
count_ab = np.array([qualificados_A, qualificados_B])
nobs_ab = np.array([leads_A, leads_B])

z_statistic_ab, p_value_ab = proportions_ztest(count_ab, nobs_ab, alternative=\'two-sided\')

print(f"\nEstatística Z: {z_statistic_ab:.4f}")
print(f"Valor P: {p_value_ab:.4f}")

alfa_ab = 0.05

if p_value_ab < alfa_ab:
    print(f"\nCom um valor P ({p_value_ab:.4f}) menor que o nível de significância ({alfa_ab}), rejeitamos a hipótese nula.")
    print("Há uma diferença estatisticamente significativa na taxa de qualificação entre os grupos A e B.")
    print("A nova estratégia (Grupo B) parece ter melhorado a qualificação de Leads.")
else:
    print(f"\nCom um valor P ({p_value_ab:.4f}) maior que o nível de significância ({alfa_ab}), não rejeitamos a hipótese nula.")
    print("Não há evidências suficientes para afirmar uma diferença estatisticamente significativa na taxa de qualificação entre os grupos A e B.")

# Visualização dos resultados do Teste A/B
data_ab = pd.DataFrame({
    \'Grupo\': [\'A (Controle)\\, \'B (Variante)\\'],
    \'Taxa de Qualificação\': [taxa_qualificacao_A, taxa_qualificacao_B]
})

plt.figure(figsize=(7, 5))
sns.barplot(x=\'Grupo\', y=\'Taxa de Qualificação\', data=data_ab, palette=\'coolwarm\')
plt.title(\'Teste A/B: Taxa de Qualificação de Leads\')
plt.ylabel(\'Taxa de Qualificação\')
plt.ylim(0, max(taxa_qualificacao_A, taxa_qualificacao_B) * 1.1)

for index, row in data_ab.iterrows():
    plt.text(index, row[\'Taxa de Qualificação\'] + 0.005, f\'{row[\'Taxa de Qualificação\']:.2%}\'
, color=\'black\', ha=\'center\')

plt.grid(axis=\'y\', linestyle=\'--\', alpha=0.7)
plt.show()

## Entrega

Siga esses passos para entregar sua prova.

1. Crie um repositório no GitHub para essa entrega.

2. Submeta seu desenvolvimento neste repositório.

3. Garanta que o repositório é publicamente acessível (eliminatório).

4. No Google forms da prova, submeta a URL do repositório.